In [ ]:
import os
import kagglehub
from google.colab import drive
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
EPOCH = 60
IMG_SIZE = 224
BATCH_SIZE = 32
path = kagglehub.dataset_download("vishalmane109/indian-currency-note-images-dataset-2020")
drive.mount('/content/drive')
model_save_path = '/content/drive/MyDrive/CurrencyRecognitionModels'

In [ ]:
class Dataset:
    def __init__(self):
        self.path = path

    def dataset(self):
        trainSet = tf.keras.preprocessing.image_dataset_from_directory(
            os.path.join(self.path, "Indian currency dataset v1/training"),
            shuffle=True,
            image_size=(IMG_SIZE, IMG_SIZE),
            batch_size=BATCH_SIZE,
        )
        valSet = tf.keras.preprocessing.image_dataset_from_directory(
            os.path.join(self.path, "Indian currency dataset v1/validation"),
            shuffle=True,
            image_size=(IMG_SIZE, IMG_SIZE),
            batch_size=BATCH_SIZE,
        )
        print("Dataset loaded successfully.")
        return trainSet, valSet

In [ ]:
class Preprocessing:
    def preprocess(self, IMG_SIZE = 224):
        resize_and_rescale = tf.keras.Sequential([
            layers.Resizing(IMG_SIZE, IMG_SIZE),
            layers.Rescaling(1.0 / 255),
        ])
        data_augmentation = tf.keras.Sequential([
            layers.RandomFlip("horizontal_and_vertical"),
            layers.RandomRotation(0.1),
        ])
        print("Preprocessing initialized.")
        return resize_and_rescale, data_augmentation

In [ ]:
class VGG16Model:
    def __init__(self, base_models):
        self.base_model = base_models
        self.base_model.trainable = False

    def build_model(self, IMG_SIZE = 224):
        prep = Preprocessing()
        resize_and_rescale, data_augmentation = prep.preprocess()
        model = Sequential([
            layers.InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 3)),
            resize_and_rescale,
            data_augmentation,
            self.base_model,
            layers.Flatten(),
            layers.Dense(1024, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(8, activation='softmax')
        ])

        dataset = Dataset()
        trainSet, valSet = dataset.dataset()


        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                      metrics=['accuracy'])

        checkpoint_cb = ModelCheckpoint(f"{model_save_path}/best_VGG16.keras",
                                        monitor='val_loss',
                                        save_best_only=True,
                                        mode='min',
                                        verbose=1)

        print("Training started...")
        trained_model = model.fit(trainSet,
                          validation_data=valSet,
                          epochs=60,
                          batch_size=BATCH_SIZE,
                          verbose=1,
                          callbacks=[checkpoint_cb])
        print("Model Trained VGG16.")
        model.save(f'{model_save_path}/VGG16.keras')

In [ ]:
class VGG19Model:
    def __init__(self, base_model):
        self.base_model = base_model
        self.base_model.trainable = True

    def build_model(self, IMG_SIZE=224):
        prep = Preprocessing()
        resize_and_rescale, data_augmentation = prep.preprocess()

        model = Sequential([
            layers.InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 3)),
            resize_and_rescale,
            data_augmentation,
            self.base_model,
            layers.Flatten(),
            layers.Dense(1024, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(8, activation='softmax')  # 8 classes for classification
        ])

        dataset = Dataset()
        trainSet, valSet = dataset.dataset()

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                      metrics=['accuracy'])

        checkpoint_cb = ModelCheckpoint(f"{model_save_path}/best_VGG19_tainable.keras",
                                        monitor='val_loss',
                                        save_best_only=True,
                                        mode='min',
                                        verbose=1)

        print("Training started...")
        trained_model = model.fit(trainSet,
                          validation_data=valSet,
                          epochs=60,
                          batch_size=BATCH_SIZE,
                          verbose=1,
                          callbacks=[checkpoint_cb])

        print("Training complete VGG19.")

In [ ]:
class ResNet50:
    def __init__(self, base_model):
        self.base_model = base_model
        self.base_model.trainable = True

    def build_model(self):
        prep = Preprocessing()
        resize_and_rescale, data_augmentation = prep.preprocess()

        model = Sequential([
            layers.InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 3)),
            resize_and_rescale,
            data_augmentation,
            self.base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            layers.Dropout(0.4),
            layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(8, activation='softmax')
        ])

        data = Dataset()
        trainSet, valSet = data.dataset()

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                      metrics=['accuracy'])

        checkpoint_cb = ModelCheckpoint(f"{model_save_path}/best_ResNet50.keras",
                                        monitor='val_loss',
                                        save_best_only=True,
                                        mode='min',
                                        verbose=1)

        print("Training started...")
        trained_model = model.fit(trainSet,
                          validation_data=valSet,
                          epochs=60,
                          batch_size=BATCH_SIZE,
                          verbose=1,
                          callbacks=[checkpoint_cb])

        print("Training complete ResNet50.")

        return trained_model

In [ ]:
class Efficientnetb0:
    def __init__(self, base_model):
        self.base_model = base_model
        self.base_model.trainable = True

    def build_model(self, IMG_SIZE=224):
        prep = Preprocessing()
        resize_and_rescale, data_augmentation = prep.preprocess(IMG_SIZE= 224)

        model = Sequential([
            layers.InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 3)),
            resize_and_rescale,
            data_augmentation,
            self.base_model,
            layers.Flatten(),
            layers.Dense(1024, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(8, activation='softmax')  # 8 classes for classification
        ])

        dataset = Dataset()
        trainSet, valSet = dataset.dataset()

        model.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                      metrics=['accuracy'])

        checkpoint_cb = ModelCheckpoint(f"{model_save_path}/best_EfficientNet.keras",
                                        monitor='val_loss',
                                        save_best_only=True,
                                        mode='min',
                                        verbose=1)

        print("Training started...")
        trained_model = model.fit(trainSet,
                          validation_data=valSet,
                          epochs=EPOCH,
                          batch_size=BATCH_SIZE,
                          verbose=1,
                          callbacks=[checkpoint_cb])

        print("Training complete EfficientNet.")

In [ ]:
class Model_Builder:
    def __init__(self):
        self.base_models = {
            "vgg16": keras.applications.VGG16(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3)),
            "vgg19": keras.applications.VGG19(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3)),
            "resnet50": keras.applications.ResNet50(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3)),
            "efficient0": keras.applications.EfficientNetB0(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        }

    def different_models(self):
        custom_vgg16 = VGG16Model(self.base_models["vgg16"])
        model_vgg16 = custom_vgg16.build_model(IMG_SIZE)
        custom_vgg19 = VGG19Model(self.base_models["vgg19"])
        model_vgg19 = custom_vgg19.build_model(IMG_SIZE)
        custom_resnet50 = ResNet50(self.base_models["resnet50"])
        model_resnet50 = custom_resnet50.build_model()
        custom_efficientnetb0 = Efficientnetb0(self.base_models["efficient0"])
        model_efficientnetb0 = custom_efficientnetb0.build_model(IMG_SIZE)
        return model_vgg19, model_vgg16, model_resnet50, model_efficientnetb0

In [ ]:
model_builder = Model_Builder()
models = model_builder.different_models()